In [1]:
import os, sys, shutil
import subprocess
import fnmatch
import numpy as np
import glob
import pandas as pd

from helperfunctions import runAFNI, splitall

## Settings

In [4]:
# animal folder

# file_path = '/media/sf_MRIDATA/Gadospin2018/20180903/test/horizontal_scans'
file_path = '/media/sf_MRIDATA/Gadospin2018/20180903/control/horizontal_scans'


template = '/media/sf_MRIDATA/Gadospin2018/20180903/template/X25P1.nii'

ROI = '/media/sf_MRIDATA/Gadospin2018/20180903/template/ROI_horizontal.nii'

## Convert scans

In [2]:
def runAFNI(inputstring):
    print(inputstring)
    subprocess.run(inputstring,shell=True, check=True)

main_path = os.path.dirname(file_path)
runAFNI(f'Bru2 {main_path}/subject')

##  Extract ROIs

In [ ]:
def splitall(path):
    allparts = []
    while 1:
        parts = os.path.split(path)
        if parts[0] == path:  # sentinel for absolute paths
            allparts.insert(0, parts[0])
            break
        elif parts[1] == path: # sentinel for relative paths
            allparts.insert(0, parts[1])
            break
        else:
            path = parts[0]
            allparts.insert(0, parts[1])
    return allparts


# for folderName, subfolders, filenames in os.walk(file_path):
#     for file in filenames:
#         if fnmatch.fnmatch(file, '*.nii'):
#             file_path1 = glob.glob(os.path.join(folderName,file))
#             file_path1 = file_path1[0]
            
#             for horizontal_scan in horizontal_scans:
#                 if fnmatch.fnmatch(file_path1, f'*{horizontal_scan}P1.nii'):
#                     print('horizontal scan')
#                     print('extracting rois from: ', file_path1)
#                     runAFNI(f'3dROIstats -mask {file_path}/{h_ROI} -sigma {file_path1} > {folderName}/ROIdata_h{horizontal_scan}.1D')
#                 if fnmatch.fnmatch(file_path1, f'*{sagittal_scan}P1.nii'):
#                     print('sagittal scan')
#                     runAFNI(f'3dROIstats -mask {file_path}/{s_ROI} {file_path1} > {folderName}/ROIdata_s{horizontal_scan-1}.1D')

for folderName, subfolders, filenames in os.walk(file_path):
    for file in filenames:
        if fnmatch.fnmatch(file, '*.nii'):
            file_path1 = glob.glob(os.path.join(folderName,file))
            file_path1 = file_path1[0] # filename
            ID = splitall(file_path1)[-1][:-6] # scan number only
            print(ID)
#             print('extracting rois from: ', file_path1)
            runAFNI(f"3dAllineate -base {template} -source {file_path1} -prefix {file_path}/{ID}coreg -cost ls -twoblur 1 -final 'wsinc5'")
            runAFNI(f'3dROIstats -mask {ROI} -sigma {ID}coreg+orig.nii > {folderName}/ROIdata_{ID}.1D')


X10
3dAllineate -base /media/sf_MRIDATA/Gadospin2018/20180903/template/X25P1.nii -source /media/sf_MRIDATA/Gadospin2018/20180903/control/horizontal_scans/X10P1.nii -prefix /media/sf_MRIDATA/Gadospin2018/20180903/control/horizontal_scans/X10coreg -cost ls -twoblur 1 -final 'wsinc5'
3dROIstats -mask /media/sf_MRIDATA/Gadospin2018/20180903/template/ROI_horizontal.nii -sigma X10coreg.nii > /media/sf_MRIDATA/Gadospin2018/20180903/control/horizontal_scans/ROIdata_X10.1D
X12
3dAllineate -base /media/sf_MRIDATA/Gadospin2018/20180903/template/X25P1.nii -source /media/sf_MRIDATA/Gadospin2018/20180903/control/horizontal_scans/X12P1.nii -prefix /media/sf_MRIDATA/Gadospin2018/20180903/control/horizontal_scans/X12coreg -cost ls -twoblur 1 -final 'wsinc5'
3dROIstats -mask /media/sf_MRIDATA/Gadospin2018/20180903/template/ROI_horizontal.nii -sigma X12coreg.nii > /media/sf_MRIDATA/Gadospin2018/20180903/control/horizontal_scans/ROIdata_X12.1D
X17
3dAllineate -base /media/sf_MRIDATA/Gadospin2018/20180903/

## Read all ROI files

In [6]:
allfiles = glob.glob(f'{file_path}/*.1D')

frame = pd.DataFrame()
list_ = []
for file_ in allfiles:
    print(file_)
    df = pd.read_table(file_)
    list_.append(df)
frame = pd.concat(list_)

frame.columns

/media/sf_MRIDATA/Gadospin2018/20180903/test/horizontal_scans/ROIdata_X12.1D
/media/sf_MRIDATA/Gadospin2018/20180903/test/horizontal_scans/ROIdata_X14.1D
/media/sf_MRIDATA/Gadospin2018/20180903/test/horizontal_scans/ROIdata_X17.1D
/media/sf_MRIDATA/Gadospin2018/20180903/test/horizontal_scans/ROIdata_X19.1D
/media/sf_MRIDATA/Gadospin2018/20180903/test/horizontal_scans/ROIdata_X21.1D
/media/sf_MRIDATA/Gadospin2018/20180903/test/horizontal_scans/ROIdata_X24.1D
/media/sf_MRIDATA/Gadospin2018/20180903/test/horizontal_scans/ROIdata_X26.1D
/media/sf_MRIDATA/Gadospin2018/20180903/test/horizontal_scans/ROIdata_X28.1D
/media/sf_MRIDATA/Gadospin2018/20180903/test/horizontal_scans/ROIdata_X30.1D
/media/sf_MRIDATA/Gadospin2018/20180903/test/horizontal_scans/ROIdata_X32.1D


Index(['File', 'Sub-brick', 'Mean_noiseR  ', 'Sigma_noiseR', 'Mean_noiseL  ',
       'Sigma_noiseL', 'Mean_signalR  ', 'Sigma_signalR', 'Mean_signalL  ',
       'Sigma_signalL'],
      dtype='object')

In [7]:
frame['SNR'] = ( frame['Mean_signalR  ']/frame['Sigma_noiseR'] + frame['Mean_signalL  ']/frame['Sigma_noiseL'] ) / 2

del frame['Sub-brick']

frame.to_excel(f'{file_path}/output.xlsx')
frame

,File,Mean_noiseR,Sigma_noiseR,Mean_noiseL,Sigma_noiseL,Mean_signalR,Sigma_signalR,Mean_signalL,Sigma_signalL,SNR
0,/media/sf_MRIDATA/Gadospin2018/20180903/test/h...,0.000077,0.000020,0.000077,0.000020,0.002269,0.001499,0.002398,0.001168,116.675000
0,/media/sf_MRIDATA/Gadospin2018/20180903/test/h...,0.000078,0.000020,0.000078,0.000020,0.003048,0.001497,0.003331,0.001244,159.475000
0,/media/sf_MRIDATA/Gadospin2018/20180903/test/h...,0.000078,0.000021,0.000079,0.000021,0.003592,0.001412,0.003629,0.001318,171.928571
0,/media/sf_MRIDATA/Gadospin2018/20180903/test/h...,0.000080,0.000021,0.000082,0.000021,0.003777,0.001427,0.003699,0.001364,178.000000
0,/media/sf_MRIDATA/Gadospin2018/20180903/test/h...,0.000083,0.000022,0.000083,0.000022,0.003752,0.001425,0.003423,0.001288,163.068182
0,/media/sf_MRIDATA/Gadospin2018/20180903/test/h...,0.000079,0.000021,0.000079,0.000020,0.001194,0.001059,0.001070,0.001025,55.178571
0,/media/sf_MRIDATA/Gadospin2018/20180903/test/h...,0.000079,0.000020,0.000080,0.000021,0.001429,0.001317,0.001281,0.001347,66.225000
0,/media/sf_MRIDATA/Gadospin2018/20180903/test/h...,0.000080,0.000021,0.000080,0.000020,0.001396,0.001292,0.001236,0.001300,64.138095
0,/media/sf_MRIDATA/Gadospin2018/20180903/test/h...,0.000079,0.000020,0.000081,0.000020,0.001312,0.001230,0.001114,0.001153,60.650000
0,/media/sf_MRIDATA/Gadospin2018/20180903/test/h...,0.000080,0.000021,0.000082,0.000021,0.001298,0.001207,0.001023,0.001040,55.261905
